In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('dirty-id.csv', encoding = 'latin-1')
df.head()

,Date,Username,Tweet
0,2021-11-29T23:54:51.000Z,"mici626\n@bruisedbadly\nÂ·\nNov 30, 2021","bikinnya harus di domisili iya, tapi selama ad..."
1,2021-11-29T23:52:01.000Z,"snoop\n@peppreoni\nÂ·\nNov 30, 2021",Temenku weekend kemarin ada yg ke Bali sama ke...
2,2021-11-29T23:26:52.000Z,Tips Merawat Ternak\n@merawatternak\nÂ·\nNov 3...,5. Kambing ini merupakan kambing asal pulau Ba...
3,2021-11-29T23:25:33.000Z,"koko turner\n@hyuckdiphile\nÂ·\nNov 30, 2021",kokoÂ² surabaya lagi liburan di Bali
4,2021-11-29T22:54:24.000Z,Nisa Fitriyani Sailolof\n@NisaFit05651805\nÂ·\...,"5 hotel nyaman harga Rp 100 ribu-an di Kuta, B..."


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54259 entries, 0 to 54258
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Date      54259 non-null  object
 1   Username  54259 non-null  object
 2   Tweet     54259 non-null  object
dtypes: object(3)
memory usage: 1.2+ MB


In [4]:
df.drop_duplicates(inplace = True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45018 entries, 0 to 54258
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Date      45018 non-null  object
 1   Username  45018 non-null  object
 2   Tweet     45018 non-null  object
dtypes: object(3)
memory usage: 1.4+ MB


In [6]:
df.head()

,Date,Username,Tweet
0,2021-11-29T23:54:51.000Z,"mici626\n@bruisedbadly\nÂ·\nNov 30, 2021","bikinnya harus di domisili iya, tapi selama ad..."
1,2021-11-29T23:52:01.000Z,"snoop\n@peppreoni\nÂ·\nNov 30, 2021",Temenku weekend kemarin ada yg ke Bali sama ke...
2,2021-11-29T23:26:52.000Z,Tips Merawat Ternak\n@merawatternak\nÂ·\nNov 3...,5. Kambing ini merupakan kambing asal pulau Ba...
3,2021-11-29T23:25:33.000Z,"koko turner\n@hyuckdiphile\nÂ·\nNov 30, 2021",kokoÂ² surabaya lagi liburan di Bali
4,2021-11-29T22:54:24.000Z,Nisa Fitriyani Sailolof\n@NisaFit05651805\nÂ·\...,"5 hotel nyaman harga Rp 100 ribu-an di Kuta, B..."


In [7]:
import re
from nltk.tokenize import word_tokenize

In [8]:
def tweet_cleansing(tweet):
    tweet = tweet.lower()
    tweet = tweet.replace(',', '')
    tweet = re.sub('(www\.[^\s]+ | (https?://[^\s]+))', 'URL', tweet)
    tweet = re.sub('[^\w]', ' ', tweet)
    tweet = re.sub('@[^\s]+', 'AT_USER', tweet)
    tweet = re.sub('#([^\s]+)', '\1', tweet)
    tweet = tweet.strip('\'"')
    return tweet

In [9]:
df['cleaned_tweet'] = df.Tweet.apply(tweet_cleansing).values
df['cleaned_tweet'] = df.cleaned_tweet.apply(lambda x: ' '.join([j for j in x.split() if len(j)>3]))

In [10]:
df['len_words'] = df.cleaned_tweet.apply(lambda x: len(x))

In [11]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from nltk.stem import WordNetLemmatizer

In [12]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

stop_factory = StopWordRemoverFactory()
stopword = stop_factory.create_stop_word_remover()

In [13]:
kata_slang = pd.read_csv('twitter_dict.csv')

In [14]:
def replace_slang(Tweet):
    Tweet = Tweet.lower()
    res = ''
    for item in Tweet.split():
        if item in kata_slang.slang.values:
            res += kata_slang[kata_slang['slang'] == item]['formal'].iloc[0]
        else:
            res += item
        res += ' '
    return res

def tokenizerku(tweet):
    words = word_tokenize(tweet)
    tokens = []
    for i in words:
        pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
        i = pattern.sub(r"\1\1", i)
        i = i.strip('\'"?,.')
        alp = re.search(r"^[a-zA-Z0-9][a-zA-Z0-9-]*$", i)
        if(i in ['AT_USER', 'URL'] or alp is None):
            continue
        else:
            i = WordNetLemmatizer().lemmatize(i)
            tokens.append(i.lower())
    return tokens

In [15]:
df.columns

Index(['Date', 'Username', 'Tweet', 'cleaned_tweet', 'len_words'], dtype='object')

In [16]:
df['cleaned_tweet'] = df.cleaned_tweet.apply(replace_slang)
df['cleaned_tweet'] = df.cleaned_tweet.apply(lambda x : stopword.remove(x))
df['cleaned_tweet'] = df.cleaned_tweet.apply(lambda x : stemmer.stem(x))

In [17]:
df.head()

,Date,Username,Tweet,cleaned_tweet,len_words
0,2021-11-29T23:54:51.000Z,"mici626\n@bruisedbadly\nÂ·\nNov 30, 2021","bikinnya harus di domisili iya, tapi selama ad...",bikin domisili lama foto bekas siapa tempat do...,198
1,2021-11-29T23:52:01.000Z,"snoop\n@peppreoni\nÂ·\nNov 30, 2021",Temenku weekend kemarin ada yg ke Bali sama ke...,temenku weekend kemarin bal sama jogja jadi pe...,74
2,2021-11-29T23:26:52.000Z,Tips Merawat Ternak\n@merawatternak\nÂ·\nNov 3...,5. Kambing ini merupakan kambing asal pulau Ba...,kambing rupa kambing asal pulau bal hambir punah,54
3,2021-11-29T23:25:33.000Z,"koko turner\n@hyuckdiphile\nÂ·\nNov 30, 2021",kokoÂ² surabaya lagi liburan di Bali,koko surabaya libur bal,33
4,2021-11-29T22:54:24.000Z,Nisa Fitriyani Sailolof\n@NisaFit05651805\nÂ·\...,"5 hotel nyaman harga Rp 100 ribu-an di Kuta, B...",hotel nyaman harga ribu kuta bal bal kuta prom...,86


In [18]:
def tokenized_text(tweet):
    tokenized_text = tweet.split()
    return tokenized_text
df['tokens'] = df['cleaned_tweet'].apply(lambda x: tokenized_text(x))

In [19]:
df.head()

,Date,Username,Tweet,cleaned_tweet,len_words,tokens
0,2021-11-29T23:54:51.000Z,"mici626\n@bruisedbadly\nÂ·\nNov 30, 2021","bikinnya harus di domisili iya, tapi selama ad...",bikin domisili lama foto bekas siapa tempat do...,198,"[bikin, domisili, lama, foto, bekas, siapa, te..."
1,2021-11-29T23:52:01.000Z,"snoop\n@peppreoni\nÂ·\nNov 30, 2021",Temenku weekend kemarin ada yg ke Bali sama ke...,temenku weekend kemarin bal sama jogja jadi pe...,74,"[temenku, weekend, kemarin, bal, sama, jogja, ..."
2,2021-11-29T23:26:52.000Z,Tips Merawat Ternak\n@merawatternak\nÂ·\nNov 3...,5. Kambing ini merupakan kambing asal pulau Ba...,kambing rupa kambing asal pulau bal hambir punah,54,"[kambing, rupa, kambing, asal, pulau, bal, ham..."
3,2021-11-29T23:25:33.000Z,"koko turner\n@hyuckdiphile\nÂ·\nNov 30, 2021",kokoÂ² surabaya lagi liburan di Bali,koko surabaya libur bal,33,"[koko, surabaya, libur, bal]"
4,2021-11-29T22:54:24.000Z,Nisa Fitriyani Sailolof\n@NisaFit05651805\nÂ·\...,"5 hotel nyaman harga Rp 100 ribu-an di Kuta, B...",hotel nyaman harga ribu kuta bal bal kuta prom...,86,"[hotel, nyaman, harga, ribu, kuta, bal, bal, k..."


In [20]:
df = pd.DataFrame(df)
df.to_csv("id-cleaned-tweet.csv")